# Dremio Test

## Setting up dremio


## Imports

In [1]:
import os
import sys
import atoti as tt

In [2]:
import pandas as pd

Make sure you are running a version of atoti older than 0.8.10. This has been tested on 0.8.5

In [3]:
tt._version.VERSION

'0.8.11.4865.20240313153100'

### Jar Input

In [4]:
dremio_driver_jar_v10="flight-sql-jdbc-driver-10.0.0.jar"
dremio_driver_jar_v15="flight-sql-jdbc-driver-15.0.0.jar"
dremio_driver_jar_legacy="dremio-jdbc-driver-24.3.2-202401241821100032-d2d8a497.jar"

In [5]:
dremio_driver_jar=dremio_driver_jar_v15

### SQL Setup

In [6]:
schema_name='"@admin"'
db_table_name='SampleData20240104'
sql = f"SELECT * FROM {schema_name}.{db_table_name}"

### Database setup

In [7]:
username="admin"
password="admin01!"
host_name="localhost"
port="32010"
database = "Dremio"

jdbc_legacy_url=f"jdbc:dremio:direct={host_name}:31010?user={username}&password={password}&useEncryption=false"
jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&password={password}&useEncryption=false"

jdbc_driver="org.apache.arrow.driver.jdbc.ArrowFlightJdbcDriver"
jdbc_legacy_driver="com.dremio.jdbc.Driver"
atoti_table_name="table_facts"

In [8]:
os.getcwd()

'/Users/gpadya/Documents/dremio-driver/Python Dremio Demo'

## Session

In [9]:
session = tt.Session(
    port=9092,
    user_content_storage = "./Python Dremio Demo/content",
    java_options=["-Dorg.springframework.boot.logging.LoggingSystem=none"],
    extra_jars=[f"./utilities/extra_jars/{dremio_driver_jar}"],
    # app_extensions=tt.ADVANCED_APP_EXTENSION,
    # logging=tt.LoggingConfig(destination=sys.stdout)
    logging=tt.LoggingConfig(destination="./logging/server.log")
)

## Load from Dremio

In [10]:
table_facts_types= {
    "AsofDate" : tt.LOCAL_DATE,
    "CUSIP" : tt.STRING,
    "SectorCode" : tt.STRING,
    "Strategy" : tt.STRING,
    "MarketValue" : tt.DOUBLE,
    "EffectiveDuration" : tt.DOUBLE,
    "OptionAdjustedSpread" : tt.DOUBLE,
    "LegalEntity" : tt.STRING,
    "PortfolioTicker" : tt.STRING,
    "OwnershipType" : tt.STRING,
    "PercentWeight" : tt.INT
}

In [11]:
# table_facts = session.create_table(
#     atoti_table_name,
#     types=table_facts_types,
# )

# table_facts.load_sql(sql,
#                 url=jdbc_url, 
#                 driver=jdbc_driver)

In [12]:
%%time
table_facts=session.read_sql(sql,
                url=jdbc_url, 
                driver=jdbc_driver,
                table_name=atoti_table_name,
)

CPU times: user 14.4 ms, sys: 6.52 ms, total: 20.9 ms
Wall time: 13.2 s


In [13]:
table_facts.head()

,AsofDate,CUSIP,SectorCode,Strategy,MarketValue,EffectiveDuration,OptionAdjustedSpread,LegalEntity,PortfolioTicker,OwnershipType,PercentWeight
0,2024-01-03 23:00:00,91282CFV8,Peaches,Paris,17042127.4,8.809704,3.81681,GEA_BM,NEABMLCWF,Passive,1.0
1,2024-01-03 23:00:00,91282CGS4,Peaches,Paris,9406929.679,6.566652,0.76185,GEA_BM,NEABMLCWF,Active,1.0
2,2024-01-03 23:00:00,ZXW2G7229,Lemons,Paris,602163.6878,0.189888,0.0,GEA_BM,NEABMNCIM,Active,1.0
3,2024-01-03 23:00:00,91282CEM9,Peaches,Paris,11985971.75,5.803752,0.0,GEA_BM,NEABMNCIM,Active,1.0
4,2024-01-03 23:00:00,91282CEV9,Peaches,Paris,7552256.246,5.95656,0.0,GEA_BM,NEABMNCIM,Active,1.0


## Create Cube

In [14]:
cube = session.create_cube(table_facts, name="facts", mode="no_measures")
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [15]:
session.tables.schema

```mermaid
erDiagram
  "table_facts" {
    _ LocalDateTime "AsofDate"
    _ String "CUSIP"
    _ String "SectorCode"
    _ String "Strategy"
    nullable double "MarketValue"
    nullable double "EffectiveDuration"
    nullable double "OptionAdjustedSpread"
    _ String "LegalEntity"
    _ String "PortfolioTicker"
    _ String "OwnershipType"
    nullable double "PercentWeight"
  }
```


### Clean Up Hierarchies

In [16]:
# del(h['MarketValue'],h['EffectiveDuration'], h['OptionAdjustedSpread'], h['PercentWeight'])

## Add Measures

In [17]:
%%time
m["MarketValue"] = tt.agg.sum(table_facts["MarketValue"])
m["EffectiveDuration"] = tt.agg.sum(table_facts["EffectiveDuration"])
m["EffectiveDuration"] = tt.agg.sum(table_facts["EffectiveDuration"])
m["OptionAdjustedSpread"] = tt.agg.sum(table_facts["OptionAdjustedSpread"])
m["PercentWeight"] = tt.agg.sum(table_facts["PercentWeight"])

CPU times: user 2.35 ms, sys: 1.64 ms, total: 4 ms
Wall time: 79.7 ms


In [18]:
%%time
m["MV Duration"] = tt.agg.sum_product(
    table_facts["MarketValue"],table_facts["EffectiveDuration"],table_facts["PercentWeight"])

CPU times: user 3.47 ms, sys: 3.25 ms, total: 6.72 ms
Wall time: 50.3 ms


In [19]:
%%time
m["MV PercentWeight"] = tt.agg.sum_product(table_facts["MarketValue"],table_facts["PercentWeight"])

CPU times: user 2.2 ms, sys: 2.58 ms, total: 4.78 ms
Wall time: 24 ms


In [20]:
%%time
m["MV OAS"] = tt.agg.sum_product(table_facts["MarketValue"],table_facts["OptionAdjustedSpread"],table_facts["PercentWeight"])

CPU times: user 2.14 ms, sys: 2.35 ms, total: 4.49 ms
Wall time: 25.5 ms


In [21]:
%%time
m["Weighted Average Duration"] = tt.where(m["MV PercentWeight"] == 0, 0, m["MV Duration"]/m["MV PercentWeight"])

CPU times: user 4.18 ms, sys: 2.7 ms, total: 6.87 ms
Wall time: 63 ms


In [22]:
%%time
m["Weighted Average Duration OAS"] = tt.where(m["MV PercentWeight"] == 0, 0, m["MV OAS"]/m["MV PercentWeight"])

CPU times: user 1.97 ms, sys: 2.31 ms, total: 4.28 ms
Wall time: 20.3 ms


## Link to UI

In [23]:
session.link

http://localhost:9092

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.